In [1]:
import os
import sys
sys.path.append(os.path.dirname(os.path.abspath('')))

In [2]:
from utils_ import *
from gensim import models

In [3]:
val_data1_path = "../dataset/newssim-train/news0.jsonl"
val_data2_path = "../dataset/newssim-train/news1.jsonl"
val_label_path = "../dataset/newssim-train/label.txt"
test_data1_path =  "../dataset/newssim-test/news0.jsonl"
test_data2_path =  "../dataset/newssim-test/news1.jsonl"
test_label_path =  "../dataset/newssim-test/label.txt"
vocab_path = './vocab.pkl'

In [4]:
val_json_list1 = make_json_list(val_data1_path)
val_json_list2 = make_json_list(val_data2_path)
test_json_list1 = make_json_list(test_data1_path)
test_json_list2 = make_json_list(test_data2_path)

In [5]:
pos_list = ['N', 'V']
word_threshold = 50

In [6]:
vocab = make_vocab([val_json_list1, val_json_list2], word_threshold, pos_list, vocab_path)

In [7]:
(val_tf1, val_tf2), val_idf, df = get_tf_and_idf(val_json_list1, val_json_list2, vocab)
val_tf_idf1 = get_tf_idf(val_tf1, val_idf)
val_tf_idf2 = get_tf_idf(val_tf2, val_idf)
val_cos_sim = get_cosine_similarity(val_tf_idf1, val_tf_idf2)

In [8]:
(val_tf1_first, val_tf2_first), _, _ = get_tf_and_idf(val_json_list1, val_json_list2, vocab, 1)
val_tf_idf1_first = get_tf_idf(val_tf1_first, val_idf)
val_tf_idf2_first = get_tf_idf(val_tf2_first, val_idf)
val_cos_sim_first = get_cosine_similarity(val_tf_idf1_first, val_tf_idf2_first)

In [9]:
(val_tf1_first3, val_tf2_first3), _, _ = get_tf_and_idf(val_json_list1, val_json_list2, vocab, 3)
val_tf_idf1_first3 = get_tf_idf(val_tf1_first3, val_idf)
val_tf_idf2_first3 = get_tf_idf(val_tf2_first3, val_idf)
val_cos_sim_first3 = get_cosine_similarity(val_tf_idf1_first3, val_tf_idf2_first3)

In [10]:
val_nnp_tf1, val_nnp_tf2 = get_nnp_tf(val_json_list1, val_json_list2, vocab)
val_nnp_tf_idf1 = get_tf_idf(val_nnp_tf1, val_idf)
val_nnp_tf_idf2 = get_tf_idf(val_nnp_tf2, val_idf)
val_nnp_cos_sim = get_cosine_similarity(val_nnp_tf_idf1, val_nnp_tf_idf2)

In [11]:
word2vec_model = models.Word2Vec.load('../tmp_files/word2vec_256d.model')

In [12]:
np.random.seed(33)

vocab_vector = np.zeros((len(vocab), 256))

for i in range(2, len(vocab)):
    vocab_vector[i] = word2vec_model.wv[vocab.idx2word[i]]

vocab_vector[1] = 0.01*np.random.randn(256)

# vocab_vector = vocab_vector_raw*val_idf.reshape(-1,1)

val_doc1 = val_tf1@vocab_vector
val_doc2 = val_tf2@vocab_vector
val_embed_cos_sim = get_cosine_similarity(val_doc1, val_doc2)

In [13]:
val_labels = get_labels(val_label_path)
cs_threshold, _ = get_score_threshold_and_print_accuracy(val_cos_sim, val_labels)
cs_threshold_first, _ = get_score_threshold_and_print_accuracy(val_cos_sim_first, val_labels)
cs_threshold_first3, _ = get_score_threshold_and_print_accuracy(val_cos_sim_first3, val_labels)
nnp_cs_threshold, _ = get_score_threshold_and_print_accuracy(val_nnp_cos_sim, val_labels)
embed_cs_threshold, _ = get_score_threshold_and_print_accuracy(val_embed_cos_sim, val_labels)

accuracy : 0.8153
accuracy : 0.7316
accuracy : 0.7795
accuracy : 0.7725
accuracy : 0.7716


In [14]:
print_test_statistics(val_cos_sim > cs_threshold, val_labels)

accuracy : 0.8153
precision : 0.7722
recall : 0.7913
F1 score : 0.7816
TP : 9130  FN : 2408
FP : 2694  TN : 13395


In [15]:
# test phase

In [16]:
(test_tf1, test_tf2), _, _ = get_tf_and_idf(test_json_list1, test_json_list2, vocab)
test_tf_idf1 = get_tf_idf(test_tf1, val_idf)
test_tf_idf2 = get_tf_idf(test_tf2, val_idf)
test_cos_sim = get_cosine_similarity(test_tf_idf1, test_tf_idf2)

In [17]:
(test_tf1_first, test_tf2_first), _, _ = get_tf_and_idf(test_json_list1, test_json_list2, vocab, 1)
test_tf_idf1_first = get_tf_idf(test_tf1_first, val_idf)
test_tf_idf2_first = get_tf_idf(test_tf2_first, val_idf)
test_cos_sim_first = get_cosine_similarity(test_tf_idf1_first, test_tf_idf2_first)

In [18]:
(test_tf1_first3, test_tf2_first3), _, _ = get_tf_and_idf(test_json_list1, test_json_list2, vocab, 3)
test_tf_idf1_first3 = get_tf_idf(test_tf1_first3, val_idf)
test_tf_idf2_first3 = get_tf_idf(test_tf2_first3, val_idf)
test_cos_sim_first3 = get_cosine_similarity(test_tf_idf1_first3, test_tf_idf2_first3)

In [19]:
test_nnp_tf1, test_nnp_tf2 = get_nnp_tf(test_json_list1, test_json_list2, vocab)
test_nnp_tf_idf1 = get_tf_idf(test_nnp_tf1, val_idf)
test_nnp_tf_idf2 = get_tf_idf(test_nnp_tf2, val_idf)
test_nnp_cos_sim = get_cosine_similarity(test_nnp_tf_idf1, test_nnp_tf_idf2)

In [20]:
test_doc1 = test_tf1@vocab_vector
test_doc2 = test_tf2@vocab_vector
test_embed_cos_sim = get_cosine_similarity(test_doc1, test_doc2)

In [21]:
test_labels = get_labels(test_label_path)

In [22]:
print_test_statistics(test_cos_sim > cs_threshold, test_labels)

accuracy : 0.8125
precision : 0.7623
recall : 0.7455
F1 score : 0.7538
TP :  574  FN :  196
FP :  179  TN : 1051


In [23]:
print_test_statistics(test_cos_sim_first > cs_threshold_first, test_labels)

accuracy : 0.7445
precision : 0.8417
recall : 0.4143
F1 score : 0.5553
TP :  319  FN :  451
FP :   60  TN : 1170


In [24]:
print_test_statistics(test_cos_sim_first3 > cs_threshold_first3, test_labels)

accuracy : 0.7980
precision : 0.8000
recall : 0.6338
F1 score : 0.7072
TP :  488  FN :  282
FP :  122  TN : 1108


In [25]:
print_test_statistics(test_nnp_cos_sim > nnp_cs_threshold, test_labels)

accuracy : 0.7425
precision : 0.6641
recall : 0.6701
F1 score : 0.6671
TP :  516  FN :  254
FP :  261  TN :  969


In [26]:
print_test_statistics(test_embed_cos_sim > embed_cs_threshold, test_labels)

accuracy : 0.7765
precision : 0.6916
recall : 0.7571
F1 score : 0.7229
TP :  583  FN :  187
FP :  260  TN :  970
